In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import sys
from pyspark.sql import Catalog

In [2]:
spark = SparkSession \
    .builder \
    .appName("Car Sales Report") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")\
    .config("spark.sql.catalog.spark_catalog.type", "hive")\
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")\
    .config("spark.yarn.access.hadoopFileSystems", "s3a://go01-demo/")\
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")\
    .config("spark.sql.catalog.local.type","hadoop")\
    .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [3]:
data_lake_name = "s3a://go01-demo/"
s3BucketName = "s3a://go01-demo/cde-workshop/cardata-csv/"
# Your Username Here:
username = "test_user_112222_7"

In [17]:
spark.catalog.currentDatabase()

'test_user_112222_7_car_data'

In [20]:
spark.catalog.listTables('default')

[Table(name='t', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='s', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='lc_smote_subset', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='driver_stats', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='table1', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='ttable2', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employee_ofer', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='tax_auth_heb', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='results_anomaly', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='mytable', database='default', description=None, 

In [21]:
spark.catalog.listTables("test_user_112222_7_car_data")

[Table(name='car_sales', database='test_user_112222_7_car_data', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='car_installs', database='test_user_112222_7_car_data', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='experimental_motors', database='test_user_112222_7_car_data', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='customer_data', database='test_user_112222_7_car_data', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='geo_data_xref', database='test_user_112222_7_car_data', description=None, tableType='EXTERNAL', isTemporary=False)]

In [25]:
#---------------------------------------------------
#               LOAD ICEBERG TABLES AS DATAFRAMES
#---------------------------------------------------

car_sales_df = spark.sql("SELECT * FROM spark_catalog.{}_CAR_DATA.CAR_SALES".format(username))
customer_data_df = spark.sql("SELECT * FROM spark_catalog.{}_CAR_DATA.CUSTOMER_DATA".format(username))

In [26]:
car_sales_df.show()

+-----------+-------+---------+------------+----+---+-----+
|customer_id|  model|saleprice|         VIN|year|day|month|
+-----------+-------+---------+------------+----+---+-----+
|        757|Model D| 86816.33|FRAWJ3W1V7VD|2020|  1|    5|
|       3440|Model R|104767.17|98ARQ2NYWFTU|2020|  1|    5|
|       6835|Model R|103947.16|2DJE665WEAR0|2020|  1|    5|
|       7987|Model C| 76069.49|O6ZF8J9BK68E|2020|  1|    5|
|       6841|Model R|107196.14|VPA418HA97H4|2020|  1|    5|
|       3124|Model C| 76169.67|MWF83R4HEMP7|2020|  1|    5|
|       7130|Model C| 69538.07|8MY2WYOF0BRW|2020|  1|    5|
|       9337|Model C| 65121.16|QO02D43Q3H7A|2020|  1|    5|
|       2132|Model R|100351.17|KJSAII9JSE5U|2020|  1|    5|
|       3704|Model R|114747.34|JUM025D8Y75H|2020|  1|    5|
|       8369|Model C| 75296.86|N66WIFFGTIGY|2020|  1|    5|
|       5269|Model C| 70467.71|XTIFDI9HPQG7|2020|  1|    5|
|       4784|Model D| 91009.55|NLT0YN474WGU|2020|  1|    5|
|       5935|Model C| 77921.76|3PFV4R55S

In [15]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [16]:
spark.sql("USE spark_catalog.{}_CAR_DATA".format(username))

Hive Session ID = b9d47c43-f1ef-42ae-be99-c27c1117b893


DataFrame[]

In [18]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------------+--------------------+
|      catalog|           namespace|
+-------------+--------------------+
|spark_catalog|test_user_112222_...|
+-------------+--------------------+



In [10]:
#spark.sql("SELECT * FROM spark_catalog.{}_CAR_DATA.CAR_SALES.history".format(username)).show()

In [11]:
# ICEBERG TABLE HISTORY (SHOWS EACH SNAPSHOT AND TIMESTAMP)
print("SHOW ICEBERG TABLE HISTORY")
print("SELECT * FROM spark_catalog.{}_CAR_DATA.CAR_SALES.history;".format(username))
#spark.sql("SELECT * FROM CAR_SALES.partitions;".format(username)).show()

SHOW ICEBERG TABLE HISTORY
SELECT * FROM spark_catalog.test_user_112222_7_CAR_DATA.CAR_SALES.history;


In [27]:
spark.sql("SELECT * FROM spark_catalog.test_user_112222_7_car_data.car_sales.history;")

AnalysisException: spark_catalog requires a single-part namespace, but got [test_user_112222_7_car_data, car_sales]